Chatbot com memoria e template com variaveis

In [1]:
# Carregar a chave de API do Google
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
_ = os.getenv('GOOGLE_API_KEY')

In [2]:
# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# bibliotecas para adicionar histórico de mensagens ao chatbot
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [3]:
# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

In [4]:
# biblioteca para passar templates de contexto
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente útil. Responda a todas as perguntas da melhor forma possível em {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [5]:
# bibliotecas para criar uma classe de estado e enviar variáveis para o chatbot
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


In [6]:

# Define a new graph
workflow = StateGraph(state_schema=State)

# Define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread_id"}}

In [7]:

query = "Iai macho, tudo suave?"
language = "Português Cearense"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language": language}, config)

output["messages"][-1].content  # output contains all messages in state

'Iai, meu rei! Tudo sussa por aqui, e contigo? Pode falar, o que tu precisa? 🤠'

In [8]:
output

{'messages': [HumanMessage(content='Iai macho, tudo suave?', additional_kwargs={}, response_metadata={}, id='1564d955-3307-46fb-865f-b03b3e9ef777'),
  AIMessage(content='Iai, meu rei! Tudo sussa por aqui, e contigo? Pode falar, o que tu precisa? 🤠', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-88c5c82e-1145-4356-b54a-702abe11a954-0', usage_metadata={'input_tokens': 28, 'output_tokens': 26, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}})],
 'language': 'Português Cearense'}

In [ ]:
query = "Transcreva o audio para texto"

# Lendo o conteúdo do arquivo local
with open("204.wav", "rb") as audio_file:
    audio_data = audio_file.read()

input_messages = HumanMessage(
    content=[
        {
            "type": "text",	
            "text": query,
        },
        {
            "type": "media",
            "data": audio_data,  # Usando o conteúdo do arquivo
            "mime_type": "audio/wav",  # Adicionando o mime_type
        },
    ]
)
output = model.invoke([input_messages])

In [25]:
output.content

'A primeira maior guerra de todas foi entre o bem e o mal, o céu e a terra.'

In [29]:
query = "Transcreva o audio para texto e cite de onde é a origem desse texto"

# Lendo o conteúdo do arquivo local
with open("206.wav", "rb") as audio_file:
    audio_data = audio_file.read()

input_messages = HumanMessage(
    content=[
        {
            "type": "text",	
            "text": query,
        },
        {
            "type": "media",
            "data": audio_data,  # Usando o conteúdo do arquivo
            "mime_type": "audio/wav",  # Adicionando o mime_type
        },
    ]
)
output2 = model.invoke([input_messages])

In [32]:
output2.pretty_print()

================================== Ai Message ==================================

O áudio contém a frase: "Eu prefiro ser essa metamorfose ambulante."

A origem dessa frase é a música "Metamorfose Ambulante" de Raul Seixas, lançada em 1973.
